In [1]:
import geopandas as gpd
import matplotlib.pyplot as plot
from shapely import geometry
from shapely.geometry import LineString, Point, box
import geopandas as gpd
import pandas as pd
import itertools
from collections import deque,namedtuple
import numpy as np
import concurrent.futures
from tqdm import tqdm

# prepare data

In [2]:
move=pd.read_csv('../data/traj data/move_relocation_rango.csv')
move.head(5)

,who,stime,etime,date,poi_id_o,poi_id_d,r,t,mode,lambda_o,phi_o,lambda_d,phi_d
0,69540422,2019-01-01 16:38:48,2019-01-01 16:58:44,20190101,0,5.0,12419,1196,4,113.842141,22.592980,113.935203,22.520286
1,69540422,2019-06-09 11:59:48,2019-06-09 12:30:52,20190609,0,1.0,4596,1864,4,113.909429,22.575772,113.942054,22.548366
2,69540422,2019-06-15 11:56:13,2019-06-15 12:26:25,20190615,0,1.0,4596,1812,4,113.909429,22.575772,113.942054,22.548366
3,69540422,2019-06-18 08:15:08,2019-06-18 08:27:48,20190618,0,1.0,4242,760,4,113.909429,22.575772,113.942054,22.548366
4,69540422,2019-06-11 12:03:13,2019-06-11 12:36:30,20190611,0,1.0,4596,1997,4,113.909429,22.575772,113.942054,22.548366


In [ ]:
routes = gpd.read_file('./data/nanshan_traj.shp')
routes=routes.to_crs('EPSG:4326')
routes = routes[routes['age'] != 16]
district=gpd.read_file('./data/nanshan_grid.shp')

In [ ]:
routes = routes.dropna(subset=['geometry'])

In [ ]:
print(set(routes['age']))
print(set(routes['gender']))

In [ ]:
young_men = routes[(routes['age'] >= 4) & (routes['age'] <= 7) & (routes['gender'] == 1)]
young_women = routes[(routes['age'] >= 4) & (routes['age'] <= 7) & (routes['gender'] == 2)]
middle_men =  routes[(routes['age'] >= 8) & (routes['age'] <= 13) & (routes['gender'] == 1)]
middle_women = routes[(routes['age'] >= 8) & (routes['age'] <= 13) & (routes['gender'] == 2)]
old_men = routes[(routes['age'] >= 14) & (routes['gender'] == 1)]
old_women = routes[(routes['age'] >= 14) & (routes['gender'] == 2)]

In [ ]:
print(len(young_men),len(young_women),len(middle_men),len(middle_women),len(old_men),len(old_women))

# Function

In [ ]:
def getFnidByPoint(first_point, second_point):
    fnid_list = []

    # Create LineString from first_point to second_point
    line = LineString([first_point, second_point])
    line_series = gpd.GeoSeries([line], crs=4326)
    line_utm = line_series.to_crs(32650)

    dist = line_utm.length.iloc[0]
    delta = int(dist // 50)

    x_diff = second_point[0] - first_point[0]
    y_diff = second_point[1] - first_point[1]
    x_values = np.linspace(first_point[0], second_point[0], delta + 1)
    y_values = np.linspace(first_point[1], second_point[1], delta + 1)
    interpolation_points = [Point(x, y) for x, y in zip(x_values, y_values)]

    # Perform overlay analysis for each point
    intersect = gpd.overlay(district, gpd.GeoDataFrame(geometry=interpolation_points, crs=4326), how='intersection', keep_geom_type=False)
    fnid_list.extend(intersect['fnid'])

    return fnid_list

In [ ]:
def routeToFnid(routes):
    def process_route(route):
        geometry_ = route
        q = deque(geometry_.coords)
        first_point = q.popleft()
        states = set()  # 使用集合来存储状态，以去重
        while q:
            second_point = q.popleft()
            fnid_list = getFnidByPoint(first_point, second_point)
            if fnid_list:
                states.update(fnid_list)
            first_point = second_point
        return list(states)

    print('the length of routes is {}'.format(len(routes)))
    routes_states = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(process_route, routes.geometry), total=len(routes)))
        for i, states_unique in enumerate(results):
            routes_states.append(states_unique)

    routes_states = np.array(routes_states)
    return routes_states

In [ ]:
actions = [0, 1, 2, 3, 4]
dirs = {0: 'r', 1: 'l', 2: 'd', 3: 'u', 4: 's'}
Step=namedtuple('Step',['state','action'])

def getActionOfStates(route_state):
    state_action = []
    length = len(route_state)

    if length == 1:
        step = Step(state=route_state[0], action=4)
        state_action.append(step)
        return state_action

    diff = np.diff(route_state) 
    def getAction(diff_value):
        if diff_value == 1:
            return 0
        elif diff_value == -1:
            return 1
        elif diff_value == 357:
            return 3
        elif diff_value == -357:
            return 2
        else:
            return 4  

    actions = np.vectorize(getAction)(diff) 

    for i, action in enumerate(actions):
        step = Step(state=route_state[i], action=action)
        state_action.append(step)

    step = Step(state=route_state[-1], action=4)
    state_action.append(step)

    return state_action

In [ ]:
import random

# If the amount of data is too large, downsampling can be performed
def randomSelectLines(geodataframe, percent_to_select=0.7):
    total_lines = len(geodataframe)
    if total_lines<1000:
        num_lines_to_select = int(total_lines * 1)
        random_line_indices = random.sample(range(total_lines), num_lines_to_select)
        randomly_selected_lines = geodataframe.iloc[random_line_indices]
        return randomly_selected_lines

    num_lines_to_select = int(total_lines * (percent_to_select))
    random_line_indices = random.sample(range(total_lines), num_lines_to_select)
    randomly_selected_lines = geodataframe.iloc[random_line_indices]
    return randomly_selected_lines

In [ ]:
def routeToTuple(routes,save_file):
    routes=randomSelectLines(routes)
    routes_states=routeToFnid(routes)
    state_action_tuple=[]
    for route_state in tqdm(routes_states):
        sta_act=getActionOfStates(route_state)
        state_action_tuple.append(sta_act)
    print(state_action_tuple[0])
    state_action_tuple=np.array(state_action_tuple)
    np.save(save_file,state_action_tuple) 

# main function

In [ ]:
routeToTuple(young_men,'./data/routes_states/0_0_states_tuple.npy')

In [ ]:
import numpy as np 
np.load('./data/routes_states/1_2_states_tuple.npy',allow_pickle=True)